### Importation de bibliotheque

In [1]:
import sys
import warnings
if not sys.warnoptions:
    warnings.simplefilter("ignore")
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [2]:
import os
import joblib
import itertools
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf
import matplotlib.pyplot as plt
from keras.callbacks import EarlyStopping
from tensorflow.keras import layers, models
from sklearn.metrics import confusion_matrix
from keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam,RMSprop 
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler, OneHotEncoder

2024-03-22 00:29:53.473891: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-22 00:29:55.727207: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-11.8/lib64:/usr/local/cuda-11.8/lib64:/usr/local/cuda-11.8/lib64:/usr/local/cuda-11.8/lib64:/home/hassouni/miniconda3/envs/tf/lib/
2024-03-22 00:29:55.727398: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; 

### Modeling

##### 1. Data preprocessing

In [3]:
df = pd.read_csv('features_mfcc.csv')
df.shape

(48648, 2161)

In [4]:
# Fill NaN with 0
df = df.fillna(0)
print(df.isna().any())

0         False
1         False
2         False
3         False
4         False
          ...  
2156      False
2157      False
2158      False
2159      False
labels    False
Length: 2161, dtype: bool


In [5]:
df.head(10)

,0,1,2,3,4,5,6,7,8,9,...,2151,2152,2153,2154,2155,2156,2157,2158,2159,labels
0,-836.056152,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,surprise
1,-570.003487,1.197243,1.255092,-3.564422,-0.638827,0.623328,1.387042,2.897087,2.724433,4.172999,...,-1.873411,-0.439743,-7.706072,-1.822334,-1.674423,-3.355158,3.119024,2.278032,-1.538715,surprise
2,-881.584656,5.148536,4.076508,2.493722,0.659431,-1.143967,-2.665554,-3.728205,-4.249881,-4.241667,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,surprise
3,-648.940197,-3.547080,2.761682,-0.540265,1.560255,-4.287701,-0.708513,4.310824,-1.258069,-3.188684,...,-3.346332,-7.575748,-3.131421,-7.384761,-2.084824,3.562538,0.524190,2.702980,-0.325480,surprise
4,-996.581726,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,-1.461556,-4.752707,-3.807003,1.825411,1.365022,-2.007692,-1.409204,0.004334,-0.162669,neutral
5,-945.669039,-7.326480,-0.611619,-1.920115,5.322925,7.815470,4.174310,0.507270,-6.471173,2.429726,...,1.054371,3.160124,0.312156,-2.149277,-1.622166,-1.206294,-1.089394,3.353752,6.404018,neutral
6,-1002.221802,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,-1.216436,-1.478790,-1.301510,-0.933480,-0.796525,-0.900837,-0.817926,-0.291825,0.353377,neutral
7,-767.600134,-8.968201,0.034676,2.169500,0.983470,6.765730,8.419183,5.383916,-1.355246,4.527593,...,2.109602,-6.161078,-7.560592,1.061182,-4.295215,-3.844797,1.952130,0.310883,-3.331528,neutral
8,-525.304260,-11.348392,-0.997466,7.913282,-8.708626,4.971757,-0.066018,-6.612823,-0.334041,1.682047,...,-2.800925,-20.028423,-22.917688,-6.553454,1.511625,-7.332297,-8.524702,4.480391,10.784654,angry
9,-302.299783,-4.596812,4.613360,2.343106,-2.371080,-4.849342,-9.014901,-3.580502,0.474071,1.461993,...,2.836169,-2.604647,-7.062410,-5.832411,-2.830796,0.058943,-4.812131,-1.685947,5.544261,angry


In [6]:
X = df.drop(labels="labels", axis=1)
Y = df["labels"]

In [7]:
encoder = OneHotEncoder()
Y = encoder.fit_transform(np.array(Y).reshape(-1,1)).toarray()

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, random_state=42, test_size=0.2, shuffle=True)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((38918, 2160), (9730, 2160), (38918, 7), (9730, 7))

In [9]:
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, random_state=42, test_size=0.1, shuffle=True)
X_train.shape, X_test.shape, X_val.shape, y_train.shape, y_test.shape, y_val.shape

((35026, 2160), (9730, 2160), (3892, 2160), (35026, 7), (9730, 7), (3892, 7))

In [10]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
X_val = scaler.transform(X_val)
X_train.shape, X_test.shape, X_val.shape, y_train.shape, y_test.shape, y_val.shape

((35026, 2160), (9730, 2160), (3892, 2160), (35026, 7), (9730, 7), (3892, 7))

In [11]:
joblib.dump(scaler,'scaler.pkl')

['scaler.pkl']

In [12]:
# We have to use 1-dimensional CNN which need specifical shape:
X_train = np.expand_dims(X_train, axis=2)
X_val = np.expand_dims(X_val, axis=2)
X_test = np.expand_dims(X_test, axis=2)
X_train.shape

(35026, 2160, 1)

##### 2. Model with feature zrc

In [13]:
earlystopping = EarlyStopping(
                        monitor ="val_acc",
                        mode = 'auto', patience = 5,
                        restore_best_weights = True
                    )

In [14]:
learning_rate_reduction = ReduceLROnPlateau(
                                monitor='val_acc',
                                patience=3,
                                verbose=1,
                                factor=0.5,
                                min_lr=0.00001
                            )

In [15]:
model = models.Sequential()
model.add(layers.Conv1D(512, kernel_size=5, strides=1, padding="same", activation="relu", input_shape=(X_train.shape[1], 1)))
model.add(layers.BatchNormalization())
model.add(layers.MaxPool1D(pool_size=5, strides=2, padding="same"))

model.add(layers.Conv1D(512, kernel_size=5, strides=1, padding="same", activation="relu"))
model.add(layers.BatchNormalization())
model.add(layers.MaxPool1D(pool_size=5, strides=2, padding="same"))
model.add(layers.Dropout(0.1))


model.add(layers.Conv1D(256, kernel_size=5, strides=1, padding="same", activation="relu"))
model.add(layers.BatchNormalization())
model.add(layers.MaxPool1D(pool_size=5, strides=2, padding="same"))

model.add(layers.Conv1D(256, kernel_size=3, strides=1, padding='same', activation='relu'))
model.add(layers.BatchNormalization())
model.add(layers.MaxPooling1D(pool_size=5, strides = 2, padding = 'same'))
model.add(layers.Dropout(0.1))


model.add(layers.Conv1D(128, kernel_size=3, strides=1, padding='same', activation='relu'))
model.add(layers.BatchNormalization())
model.add(layers.MaxPooling1D(pool_size=3, strides = 2, padding = 'same'))

model.add(layers.Flatten())
model.add(layers.Dense(512, activation='relu'))
model.add(layers.BatchNormalization())
model.add(layers.Dense(7, activation="softmax"))
model.compile(optimizer="rmsprop", loss="categorical_crossentropy", metrics=["acc"])
model.summary()

2024-03-22 00:30:51.043286: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-03-22 00:30:51.058483: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-03-22 00:30:51.059133: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-03-22 00:30:51.060702: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorF

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 2160, 512)         3072      
                                                                 
 batch_normalization (BatchN  (None, 2160, 512)        2048      
 ormalization)                                                   
                                                                 
 max_pooling1d (MaxPooling1D  (None, 1080, 512)        0         
 )                                                               
                                                                 
 conv1d_1 (Conv1D)           (None, 1080, 512)         1311232   
                                                                 
 batch_normalization_1 (Batc  (None, 1080, 512)        2048      
 hNormalization)                                                 
                                                        

In [ ]:
with tf.device('/GPU:0'):
    history = model.fit(X_train, y_train, validation_data=(X_val, y_val),callbacks=[learning_rate_reduction, earlystopping],
                    epochs=50, batch_size=64)

Epoch 1/50


2024-03-22 00:31:02.871704: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:428] Loaded cuDNN version 8100
2024-03-22 00:31:07.147277: I tensorflow/compiler/xla/service/service.cc:173] XLA service 0x3017f4d0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-03-22 00:31:07.147339: I tensorflow/compiler/xla/service/service.cc:181]   StreamExecutor device (0): NVIDIA GeForce RTX 2060 with Max-Q Design, Compute Capability 7.5
2024-03-22 00:31:07.159006: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-03-22 00:31:07.438225: I tensorflow/compiler/jit/xla_compilation_cache.cc:477] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


302/548 [===============>..............] - ETA: 5:20 - loss: 1.5141 - acc: 0.4530

In [ ]:
print("Accuracy of our model on test data : " , model.evaluate(X_test,y_test)[1]*100 , "%")

fig , ax = plt.subplots(1,2)
train_acc = history.history['acc']
train_loss = history.history['loss']
test_acc = history.history['val_acc']
test_loss = history.history['val_loss']

fig.set_size_inches(20,6)
ax[0].plot(train_loss, label = 'Training Loss')
ax[0].plot(test_loss , label = 'Testing Loss')
ax[0].set_title('Training & Testing Loss')
ax[0].legend()
ax[0].set_xlabel("Epochs")

ax[1].plot(train_acc, label = 'Training Accuracy')
ax[1].plot(test_acc , label = 'Testing Accuracy')
ax[1].set_title('Training & Testing Accuracy')
ax[1].legend()
ax[1].set_xlabel("Epochs")
plt.show()

In [ ]:
# predicting on test data.
pred_test = model.predict(X_test)
y_pred = encoder.inverse_transform(pred_test)

y_test1 = encoder.inverse_transform(y_test)

In [ ]:
df = pd.DataFrame(columns=['Predicted Labels', 'Actual Labels'])
df['Predicted Labels'] = y_pred.flatten()
df['Actual Labels'] = y_test1.flatten()

df.head(10)

In [ ]:
def plot_confusion_matrix(cm, classes,
                        normalize=False,
                        title='Confusion matrix',
                        cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
            horizontalalignment="center",
            color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

In [ ]:
cm = confusion_matrix(y_test1, y_pred)
cm_plot_labels = ['angry', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise']
plot_confusion_matrix(cm=cm, classes=cm_plot_labels, title='Confusion Matrix')

In [ ]:
from sklearn.metrics import classification_report
print(f'Model Confusion Matrix\n',classification_report(y_test1,y_pred,target_names=cm_plot_labels))

In [ ]:
# save model
model.save("./model_mfcc.h5")

In [26]:
# save encoder
joblib.dump(encoder,'encoder_all.pkl')

['encoder_all.pkl']